---
# Exercises for Modern Experimental Physics III
## Exercise sheet 2: Detector simulation
---

This exercise sheet targets the general workflow understanding of detector simulation studies. Firstly, [Exercise 0](#Exercise0) provides an overview of the here used methods to access the `geant4_simulation` package which provides a user-friendly and quick frontend to the underlying `Geant4` framework. For any in-depth questions about the presented methods, do not be afraid of checking the `geant4_simulation` code. Secondly, [Exercise 1](#Exercise1) discusses in a lightweight fashion the interaction of photons and electrons with matter. Subsequently, [Exercise 2](#Exercise2) studies in-depth the generation and properties of electromagnetic showers. [Exercise 3](#Exercise3) performs a simplified calibration to connect a measured detector readout to the observed deposited energy of photons. 

---
# Exercise 0: Introduction to *python* Workflow with Geant4 <a id="Exercise0"></a>
---

The key aspects of the workflow with Geant4 in pyhton for the following exercise are implemented in the `geant4_simulation` package and made accessible to the user via the `ApplicationManager` class. First, to start the simulating events, the package needs to be loaded and the steering class `ApplicationManager` has to be constructed. For a better understanding of the following procedure, it may be advisable to study the code in `geant4_simulation/__init__.py`.

In [ ]:
import geant4_simulation as g4sim
# Create an instance of the ApplicationManager class.
g4 = g4sim.ApplicationManager()

There are three main ingredients to a simulation of the passage of particles through matter using Geant4:
1. The *description of the target*
2. The *underlying interaction*
3. The *initial state*

## 1. Target
The geometrical structure and the compounds of the simulated detector used in the exercises are already provided in the `geant4_simulation` package. Thus, for the exercises it suffices to just load the corresponding geometry with the `set_geometry()` module from the `ApplicationManager` class. All geometries, including their properties and arguments can be looked up in `geant4_simulation/geometry/`.

A simple lead geometry can be loaded from the geometry `pbbox`. As a default, the lead volume is 20 cm long and 10 cm in height and width. You can specify the dimensions with additional arguments of the `pbbox` geometry (dimensions in units of cm) where the second given parameter corresponds to the width and height of the lead volume:

In [ ]:
# Set the geometry to example_geometry.
g4.set_geometry('pbbox(2,2)')

The `pbbox` geometry creates a volume of lead as a default, but you can also switch the material to iron:
```python
g4.change_material("Fe")
```
This method will change the material from the previous material to the material given as an argument. Currently, only iron and lead are supported as materials.

## 2. Underlying Interaction

The second main ingredient of a simulation with Geant4 is the physics list. It specifies all particles and interactions considered in the simulation. It is mandatory for all simulations. While the Geant4 collaboration provides a vast amount of different lists which are useful for various applications, most experiments decide to define a specialised list for their own needs (like target properties, particle energies, physics needs,...). 

You do not need to modify any of the physics lists. For the first exercise, we will use a physics list that only defines electromagnetic interactions of the involved particles. Additional lists can be found in `geant4_simulation/physicsList/src/`.

In [ ]:
# Set the physics list used during the simulation.
g4.set_physics_list('EMPhysics')

After the used detector geometry and physics lists are defined, the Geant4 kernel can be initialized. Note that the geometry may be changed after the initialization of the kernel whereas the physics lists can only be changed before calling the `initialize()` method. To enable the changes in the geometry the `initialize()` method needs to be invoked.

In [ ]:
# Initialize the Geant4 kernel.
g4.initialize()

## 3. Initial State

Lastly, the third ingredient of the simulation is the specification of the incoming particles that hit the detector during the simulation. Throughout the exercises, we will use varying single incident particles with specified incoming energies. The type of the primary particle can be set using the MC Particle Numbering scheme defined by the Particle Data Group ([PDG](https://pdg.lbl.gov/2024/reviews/rpp2024-rev-monte-carlo-numbering.pdf)) of the requested particle.

To define the incoming particle use the `set_particle(Particle)` method of the steering class. Here, `Particle` can be any Geant4 accepted value - for this exercise the integer PDG code of a particle suffices. The initial energy of the particle can be set by the `set_energy(Value)` method where `Value` defines the energy in GeV.

In [ ]:
# Set the primary particle used in the simulation.
g4.set_particle(22) # 22 = photon
g4.set_energy(0.001)  # 1 MeV initial energy.

After defining all necessary input parameters of the simulation, the simulation can be started using the `start_run()` method. You can specify, the number of simulated events you want to produce and whether the simulated events should be visualized using the `numberOfEvents` and `visualize` keyword arguments, respectively. The default number of simulated events is 1 and by default the visualization of the event is disabled. The first invocation of the `start_run()` method will produce a large list of all defined physical processes in the physics list and the parameters of the corresponding processes.

In [ ]:
g4.start_run()

---
# Exercise 1: Interaction of photons with matter <a id="Exercise1"></a>
---

In the following exercise, we will determine the rate of different photon interactions with matter as a function of the photon energy by taking into account the bremsstrahlung of the electrons. During the exercise we are going to simulate the interaction of photons with energies up to 30 MeV and a small lead block and classify the occuring prcoesses. We will use this exercise to briefly discuss how a simulation study in Geant4 can be set up using the `geant4_simulation` interface.

As a first, purely qualitative step, study visually the interaction of photons with a $(2\times 2\times 2)\,$cm lead block. For this pupose, start runs at energies in the $\mathcal{0}(1\,\textrm{MeV}), \mathcal{0}(10\,\textrm{MeV}), \mathcal{0}(100\,\textrm{MeV}), \mathcal{0}(1\,\textrm{GeV}), \mathcal{0}(10\,\textrm{GeV})$ spectrum and try to identify all the different interactions.

**Hint:** To get a graphical representation of the simulation you have to explicitly set the corresponding option `visualize=True` when calling the `start_run()` method. To determine the type of process, the charges of the participating particles can be analysed. The colours of the tracks in the event display tell you the charge of the corresponding particle
* photon: green
* electron: blue
* positron: magenta

<div class="alert alert-info">
<strong>Question 1.1:</strong> 
What processes can you observe? Do the rates of the occurring processes change at different energies? 
</div>

<div class="alert alert-success">
<strong>Answer 1.1:</strong> 
    
*Your answer...*
</div>

In [ ]:
# Your code ...



Now, we would like to analyse the rate of different elementary processes more quantitatively. For a large number of events, it is not feasible to analyse the event displays by hand. Interactions of the initial photon for a **single event** can be extracted with the `get_first_interaction()` method of the steering class (This still holds true, even if the run is performed with more than one `numberOfEvents`). In this case, the output of `get_first_interaction()` is a set containing the leading processes as strings or in case of no interaction an empty set. In cases where there is an interaction recorded, the output can be "compt" for Compton scattering, "conv" for pair production or "phot" for the photo electric effect. There is the possibility that the initial particle has Compton scattered and then performed another interaction so the output of `get_first_interaction()` will contain more than one value.

For this analysis, write a method that performs the simulations of multiple, singular runs depending on a given energy and number of total events. This method should keep track of all the possible interactions (including none) and return the total number of each after reaching the desired number of simulations. Here, it is sufficient to first check for Compton scattering and only count for it if it occurs. 

Analyse about 50 different energies each in the intervals $[0.1,5]\,\textrm{MeV}$ and $[5,30]\,\textrm{MeV}$ each with around 1000 different events. Keep track of the different processes and plot the relative occurrence over the simulated energy (a plot with a logarithmic $x$-scale could ring a bell).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Your code ...



<div class="alert alert-info">
<strong>Question 1.2:</strong> 
Does the result match the expectation for the interaction of photons with the material?
</div>

<div class="alert alert-success">
<strong>Answer 1.2:</strong> 

*Your answer...*
</div>

<div class="alert alert-info">
<strong>Question 1.3:</strong> 
How many pair production processes do you see below 1 MeV?
</div>

<div class="alert alert-success">
<strong>Answer 1.3:</strong> 

*Your answer...*
</div>

Lastly, we would like to investigate the influence of the detector material on the interactions occurring.
For this change the material of the detector volume to iron and repeat the simulations done for the lead volume.
Estimate the rates of the occuring processes and compare them to the one obtained for lead.

In [ ]:
# Your code ...



<div class="alert alert-info">
<strong>Question 1.4:</strong> 
How do the results change? Motivate your answer with physical arguments.
</div>

<div class="alert alert-success">
<strong>Answer 1.4:</strong> 

*Your answer...*
</div>

---
# Exercise 2: Electromagnetic showers <a id="Exercise2"></a>
---

Within this exercise, we would like to investigate some properties of electromagnetic showers in dense materials. For this, we will start by determining the radiation length of the detector material from the interactions of photons with the material. The second part of the exercise will then focus on the energy profiles of electromagnetic showers and a comparison of photon and electron-induced showers is performed.

To determine the radiation length of lead $X_0^{\mathrm{Pb}}$, simulate 5'000 photons with an energy 1 GeV interacting with a lead block twice the size in each dimension of the one used in the previous exercise, i.e., $(4\times 4\times 4)\,$cm.
Change the geometry accordingly and do not forget to reinitialize the Geant4 kernel with the `initialize()` method.
Create and plot a histogram of the $x$ positions (dimension along the incoming photon) of the first interaction vertex.
To determine the position of the first vertex use the member function `get_x_of_first_vertex()` of the `ApplicationManager`. It returns the $x$ coordinate of the first vertex in cm. (Same usage as `get_first_interaction()`)

**Calculation of $X_0^{\mathrm{Pb}}$ Hint a)** : Use a good approximation to read the value directly from the histogram $x$-axis. <br>
**Calculation of $X_0^{\mathrm{Pb}}$ Hint b)** : Fit the histogram to the expected distribution of $x$ and extract the value as a fitparameter. (Useful modules via `kafe2`: `XYContainer`, `Fit`, `Plot`, `HistContainer`, `kafe2.fit.histogram.HistCostFunction_NegLogLikelihood`

<div class="alert alert-info">
<strong>Question 2.1:</strong> 
What distribution do you expect? Why? Determine the radiation length $X_{0}$ of the detector material. $X_{0}$ corresponds to the depth after which about 54 % of the photons have created an electron-positron pair. 
</div>

<div class="alert alert-success">
<strong>Answer 2.1:</strong>

*Your answer...*
</div>

<div class="alert alert-info">
<strong>Question 2.2:</strong> 
How is this number motivated?
</div>

<div class="alert alert-success">
<strong>Answer 2.2:</strong> <br>

*Your answer...*
</div>

In [ ]:
# Your code ...



In [ ]:
# Use of kafe2 for option b)
from kafe2 import XYContainer, Fit, Plot, HistContainer
from kafe2.fit.histogram import HistCostFunction_NegLogLikelihood


# define fit function
def fitfunc(x, <function arguments>):
    return <function>

# prepare fit
bin_edges = <define usefull binning here>
nbins = len(bin_edges)-1
bin_range = (bin_edges[0], bin_edges[-1])
hdat = HistContainer(nbins, bin_range, bin_edges=bin_edges, fill_data=<unbinned data array>)

hdat.label = "Vertex Data"
hdat.axis_labels= ['First Vertex Position $x$ (cm)', f'Entries / ({np.mean(np.diff(bin_edges)):.2f} cm)']

hfit = Fit(
    hdat, 
    fitfunc, 
    cost_function=HistCostFunction_NegLogLikelihood(data_point_distribution='poisson')
)

# add cosmetic options
# do not get hung up on this! Skip this, if you do not want to get (non existent) extra points for beauty
hfit.assign_parameter_latex_names(Syntax: <parameter variable defined as in `fitfunf`>=<latex expression>) # input should be keywardarguments for all function arguments
hfit.assign_model_function_latex_expression(<latex expression of formula>) # escape \ by \\ and expressions in {} need to be escaped as well, e.g. 1/2=\\frac{{1}}{{2}} or e^foo=e^{{foo}}
hfit.model_label = "Exponential Decay"

# execute fit
hfit.do_fit()

# show fit
plot = Plot(hfit)
foo = plot.plot()
plt.show()

In the second part of the exercise, we are going to analyse the process of the shower evolution for electrons with energies of up to several GeV.
First, we will have a look at the starting point of the showers.
For this, we will use the same detector geometry as in the first part of the exercise. 
Simulating 100 collisions of 1 GeV electrons and analyse the vertex of the first interaction. Compare the resulting distribution to the distribution obtained for photons in the previous part of the exercise.

In [ ]:
# Your code ...



<div class="alert alert-info">
<strong>Question 2.3:</strong> 
What is the difference between the starting point of electron and photon induced showers?
</div>  

<div class="alert alert-success">
<strong>Answer 2.3:</strong>

*Your answer...*
</div>

For the analysis of the shower depth and width, enlarge the dimensions of the lead block (about $(40\times 40\times 40)\,$cm) so that the full shower is contained in the lead block. 
Use energies of 3 GeV for the primary electrons for the analysis of the depth and width of the electromagnetic showers.

In [ ]:
# Your code ...



First, we will have a look at the distributions of the energy depositions in longitudinal ($x$-axis) and transverse ($y$-$z$-plane) directions for a single simulated event.
For this, fill two histograms with the positions of the energy depositions in each direction weighted by the energy deposited at this position.
The `get_step_energy_deposit()` method of the `ApplicationManager` class returns a list of four NumPy arrays representing the $x$, $y$, $z$ coordinates (in cm) and energy deposit (in GeV) at this point for each step of the particles through the detector.

**Hint:** Matplotlib's and NumPy's histogram modules both support costum weighting via a `weight` argument.

In [ ]:
# Your code ...



As the last step, study the shower dimension parallel and perpendicular to the flight direction of the electron.
The dimensions of a shower can be defined as the 90 % quantile of the distributions of deposited energy.
To calculate the so-defined dimension, repeat the previous exercise and calculate at which energy-weighted position the 90 % quantile is reached.

Simulate 500 events and calculate the shower dimensions. Histogram the results and calculate the mean of each distribution.

<div class="alert alert-info">
<strong>Question 2.4:</strong> 
How many $X_{0}$ are sufficient to contain at least 90 % of the shower longitudinally?
</div>  

<div class="alert alert-success">
<strong>Answer 2.4:</strong>

*Your answer...*
</div>

<div class="alert alert-info">
<strong>Question 2.5:</strong> 
How many radiation lengths are typically used for electromagnetic calorimeters in detectors like the CMS detector?
</div>  

<div class="alert alert-success">
<strong>Answer 2.5:</strong>

*Your answer...*
</div>

<div class="alert alert-info">
<strong>Bonus question:</strong> 
What is the transverse shower shape? How many $X_0$ are needed to contain the shower?
</div>  

<div class="alert alert-success">
<strong>Answer Bonus:</strong>
    
*Your answer...*
</div>

---
# Calorimeter <a id="Intro"></a>
---
In the lectures two specialised calorimeter concepts with the common task of measuring the energy of incoming particles were introduced.
The distinction between these two types is often made because of the respective challenges they face:
- Electromagnetic calorimeters are primarily optimised to measure as precisely as possible the energy of photons and electrons.

In the previous exercises, you were already made familiar with the basic concepts of this type of shower.
The other type of calorimeters, namely hadron calorimeters, specials on dealing with hadronic showers.
The key differences for hadronic showers are:

- The hadronic interaction length is much larger than the radiation length of photons and electrons.
- Hadronic showers consist to about a third of neutral pions that decay into two photons.
- Lastly, hadronic showers contain a large fraction (20-40%) of "invisible" particles which reduces the possible energy resolution in comparison to electromagnetic calorimeters. 

<div>
<img src="HadronicShower.png" width="800"/>
</div>

[Source](https://www.physi.uni-heidelberg.de/~sma/teaching/ParticleDetectors2/sma_HadronicCalorimeters.pdf)


Another design choice for calorimeters is (are) the used material(s). A first-order distinction can be made for the homogenous and sampling calorimeter:
- **Sampling calorimeter** <br>
    Sampling calorimeters have a layered structure with alternating layers of absorber (passive) and detector (active) material.
    On the one hand, the design and material choice are flexible and optimisable to a specific task or financial budget.
    On the other hand, some of the energy is deposited in the absorber which limits the energy resolution.
- **Homogenous calorimeter** <br>
    Here, one material is used simultaneously as an absorber and active material.
    Consequently, all the energy is deposited in the detector which allows for an optimal energy resolution.
    Also, the choice of the material can be specialised to the task, e.g. CsI has a very low light yield but allows for fast readout; CsI(Tl) has high light yield but slow readout. 
    However, the materials capable of this task (mostly scintillator crystals) are custom made which makes them often very expensive and very heavy.
    
The following exercises will focus on a sampling calorimeter structure with lead absorbers and liquid argon as active material.

A key aspect of calorimeters is the readout of the detector which determines the measured energy of the particle.
As discussed in the lecture the total number of particles in an electromagnetic shower is proportional to the initial energy of the particle.
One very common use case in high-energy particle experiments is the use of scintillation materials.
In this case, the electromagnetic shower leads to the production of photons in the visible spectrum which can be measured by photo multipliers, photo diodes, etc.
For example, the voltage measured at a silicon photomultiplier is proportional to the number of incoming photons which in turn is proportional to the initial energy.
The following exercises will discuss a closely related measurement principle: The count of charged particles in the active material.

---
# Exercise 3: Calorimeter calibration <a id="Exercise3"></a>
---

This exercise presents a very simplified version of the calibration of the photon energy deposition in a calorimeter. 
The goal is to determine the relation between the number of charged shower particles (in reality this would be a current) in the detector and the corresponding initial energy.
The setup is inspired by the [sampling electromagnetic calorimeter (ECAL) of ATLAS](https://atlas.cern/Discover/Detector/Calorimeter).
Here, incoming particles can shower in the lead absorbers.
The shower particles then ionize the liquid argon, ions drift to electrodes and the resulting number of charged particles (current) is measured.

To set up the simulation for this exercise follow the initialization instructions from the previous exercises.
In this and the following exercises, we will use a different physics list than in the previous exercises. 
We will use the physics list `MyQGSP_BERT` which includes hadronic interactions as well.

The desired sampling calorimeter can be built by the `samplingcalo(lenAbsorber, lenActive, numLayers)` geometry.
This method produces `numLayers` layers of $(10\times 10\times\text{lenAbsrober/lenActive})\,$cm alternating absorber/active material. <br>
For example,
```python
  g4.set_geometry('samplingcalo(2.,1.,15)')
```
will create a calorimeter with lead absorbers of thickness 2 cm and scintillator tiles of thickness 1 cm, and a total of 15 absorber layers.

Initialize the simulation:
- build the calorimeter from the example,
- set the correct physics list and
- initialize.

In [ ]:
import geant4_simulation as g4sim
import numpy as np
import matplotlib.pyplot as plt
from kafe2 import XYContainer, Fit, Plot

# Initialize the simulation.
# Your code ...


In this exercise, calibrate the given calorimeter for photons, i.e. determine the constant that translates the number $N_{c}$ of charged particles passing the scintillator into the energy $E_{\text{in}}$ of the initial particle.

To compute $N_{c}$, simulate for 15 different energies in the interval of $[1, 15]\,$GeV the interaction of 10 photons with the given calorimeter.
For each simulated photon, read the number of charged particles traversing the scintillator layers. 
The number can be obtained using the `calo_readout()` method from the `ApplicationManager` class. 
Be aware, this method only works for one event, therefore, you need to loop over the desired number of photon interaction simulations.
Compute the mean and its uncertainty (standard deviation divided by the square root of the number of samples) for each energy and store the result.
Finally, fit the mean number of charged tracks per event and the simulated energy with an appropriate model.

**Hint**: A smaller sample size while debugging the code can be useful here. <br>
**Hint**: For a fit with `kafe2` you can use the `XYContainer` and `Fit` modules.

In [ ]:
# Your code ...
